Напишем скрипты для тренировки моделей.

Сделаем необходимые импорты для работы в Google Colab.

In [1]:
from google.colab import drive
from google.colab import files

Монтируем Google Диск в виртуальную машину Google Colab.

In [2]:
drive.mount('/GD',force_remount=True)

Mounted at /GD


Копируем на диск архив с фото и распакуем его.

In [ ]:
!cp -r /GD/'My Drive'/dataset.zip .
!unzip dataset.zip

Импортируем keras для сборки моделей

In [4]:
import tensorflow as tf

from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Размер входного изображения.

In [5]:
Pic_Y, Pic_X = 126, 224

Подготовим генераторы для train и test множеств.

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255)
                            
train_generator = train_datagen.flow_from_directory('to_train',
                                                   target_size = (Pic_Y, Pic_X),
                                                   batch_size = 64,
                                                   class_mode = 'categorical')

Found 81341 images belonging to 9 classes.


In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)
                            
test_generator = test_datagen.flow_from_directory('to_test',
                                                   target_size = (Pic_Y, Pic_X),
                                                   batch_size = 64,
                                                   class_mode = 'categorical')

Found 18690 images belonging to 9 classes.


Соберем модель.

In [ ]:
model = models.Sequential()

model.add(layers.Conv2D(filters=32, kernel_size=3, activation="relu", 
                        input_shape = (Pic_Y, Pic_X, 3)))
model.add(layers.MaxPooling2D(pool_size=2))

model.add(layers.Conv2D(filters=64, kernel_size=3, activation="relu"))
model.add(layers.MaxPooling2D(pool_size=2))

model.add(layers.Conv2D(filters=128, kernel_size=3, activation="relu"))
model.add(layers.MaxPooling2D(pool_size=2))

model.add(layers.Conv2D(filters=256, kernel_size=3, activation="relu"))
model.add(layers.MaxPooling2D(pool_size=2))
model.add(layers.Conv2D(filters=256, kernel_size=3, activation="relu"))

model.add(layers.Flatten())
model.add(layers.Dense(9, activation="softmax"))


In [ ]:
model.compile(loss="categorical_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 124, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 62, 111, 32)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 60, 109, 64)       18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 30, 54, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 52, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 14, 26, 128)      0

In [ ]:
callbacks = [tf.keras.callbacks.ModelCheckpoint(
        filepath="convnet_from_scratch.h5",
        save_best_only=True,
        monitor="val_accuracy")]

In [ ]:
history = model.fit(train_generator, epochs=20, validation_data=test_generator,
    callbacks=callbacks)


1/20 loss: 1.2831, accuracy: 0.5434, val_loss: 0.5810, val_accuracy: 0.8075

2/20 loss: 0.5314, accuracy: 0.8242, val_loss: 0.3469, val_accuracy: 0.8899

3/20 loss: 0.3470, accuracy: 0.8886, val_loss: 0.4122, val_accuracy: 0.8821

4/20 loss: 0.2607, accuracy: 0.9182, val_loss: 0.2809, val_accuracy: 0.9212

5/20 loss: 0.2139, accuracy: 0.9353, val_loss: 0.2775, val_accuracy: 0.9268

6/20 loss: 0.1758, accuracy: 0.9464, val_loss: 0.3644, val_accuracy: 0.9051

7/20 loss: 0.1531, accuracy: 0.9547, val_loss: 0.3185, val_accuracy: 0.9257

8/20 loss: 0.1424, accuracy: 0.9600, val_loss: 0.3154, val_accuracy: 0.9292

9/20 loss: 0.1313, accuracy: 0.9644, val_loss: 0.6428, val_accuracy: 0.9056

10/20 loss: 0.1329, accuracy: 0.9652, val_loss: 0.4112, val_accuracy: 0.9269

11/20 loss: 0.1246, accuracy: 0.9681, val_loss: 0.4633, val_accuracy: 0.9234

12/20 loss: 0.1258, accuracy: 0.9702, val_loss: 0.6534, val_accuracy: 0.9169

13/20 loss: 0.1195, accuracy: 0.9718, val_loss: 0.6136, val_accuracy: 0.9165

14/20 loss: 0.1203, accuracy: 0.9731, val_loss: 0.5709, val_accuracy: 0.9261

15/20 loss: 0.1221, accuracy: 0.9742, val_loss: 0.6882, val_accuracy: 0.9224

16/20 loss: 0.1303, accuracy: 0.9745, val_loss: 0.6877, val_accuracy: 0.9246

17/20 loss: 0.1300, accuracy: 0.9761, val_loss: 0.7353, val_accuracy: 0.9307

18/20 loss: 0.1342, accuracy: 0.9762, val_loss: 0.8583, val_accuracy: 0.9255

19/20 loss: 0.1342, accuracy: 0.9767, val_loss: 0.9406, val_accuracy: 0.9265

20/20 loss: 0.1305, accuracy: 0.9772, val_loss: 0.8778, val_accuracy: 0.9305

Сохраним модель на Google Disk.

In [ ]:
!cp convnet_from_scratch.h5 /GD/'My Drive'/convnet_from_scratch.h5

convnet_from_scratch.h5  dataset.zip  sample_data  to_test  to_train  to_val


Данный генератор будет случайным образом искажать обучающие фотографии

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range = 25,
                                   shear_range = 0.15,
                                   zoom_range = 0.3,
                                   fill_mode = 'nearest')
                            
train_generator = train_datagen.flow_from_directory('to_train',
                                                   target_size = (Pic_Y, Pic_X),
                                                   batch_size = 64,
                                                   class_mode = 'categorical')

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)
                            
test_generator = test_datagen.flow_from_directory('to_test',
                                                   target_size = (Pic_Y, Pic_X),
                                                   batch_size = 64,
                                                   class_mode = 'categorical')

Заново соберем ту же модель, но обучим ее на аугментированных данных

In [ ]:
model = models.Sequential()

model.add(layers.Conv2D(filters=32, kernel_size=3, activation="relu", 
                        input_shape = (Pic_Y, Pic_X, 3)))
model.add(layers.MaxPooling2D(pool_size=2))

model.add(layers.Conv2D(filters=64, kernel_size=3, activation="relu"))
model.add(layers.MaxPooling2D(pool_size=2))

model.add(layers.Conv2D(filters=128, kernel_size=3, activation="relu"))
model.add(layers.MaxPooling2D(pool_size=2))

model.add(layers.Conv2D(filters=256, kernel_size=3, activation="relu"))
model.add(layers.MaxPooling2D(pool_size=2))
model.add(layers.Conv2D(filters=256, kernel_size=3, activation="relu"))

model.add(layers.Flatten())
model.add(layers.Dense(9, activation="softmax"))


In [ ]:
model.compile(loss="categorical_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 124, 222, 32)      896       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 62, 111, 32)      0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 60, 109, 64)       18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 30, 54, 64)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 28, 52, 128)       73856     
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 14, 26, 128)     

In [ ]:
callbacks = [tf.keras.callbacks.ModelCheckpoint(
        filepath="augmentation_model.h5",
        save_best_only=True,
        monitor="val_loss")]

In [ ]:
history = model.fit(train_generator, epochs=20, validation_data=test_generator,
    callbacks=callbacks)

1/20 loss: 1.5435 - accuracy: 0.4434 - val_loss: 0.9150 - val_accuracy: 0.6844

2/20 loss: 0.9536 - accuracy: 0.6729 - val_loss: 0.5941 - val_accuracy: 0.8146

3/20 loss: 0.7394 - accuracy: 0.7494 - val_loss: 0.4865 - val_accuracy: 0.8523

4/20 loss: 0.6412 - accuracy: 0.7889 - val_loss: 0.3856 - val_accuracy: 0.8815

5/20 loss: 0.5808 - accuracy: 0.8126 - val_loss: 0.4446 - val_accuracy: 0.8842

6/20 loss: 0.5589 - accuracy: 0.8210 - val_loss: 0.4405 - val_accuracy: 0.8833

7/20 loss: 0.5356 - accuracy: 0.8284 - val_loss: 0.3989 - val_accuracy: 0.8811

8/20 loss: 0.5285 - accuracy: 0.8352 - val_loss: 0.3570 - val_accuracy: 0.9022

9/20 loss: 0.5221 - accuracy: 0.8385 - val_loss: 0.3765 - val_accuracy: 0.8951

10/20 loss: 0.5252 - accuracy: 0.8369 - val_loss: 0.3812 - val_accuracy: 0.8919

11/20 loss: 0.5328 - accuracy: 0.8364 - val_loss: 0.4026 - val_accuracy: 0.8930

12/20 loss: 0.5503 - accuracy: 0.8346 - val_loss: 0.4290 - val_accuracy: 0.8860

13/20 loss: 0.5667 - accuracy: 0.8302 - val_loss: 0.3573 - val_accuracy: 0.9120

14/20 loss: 0.5895 - accuracy: 0.8254 - val_loss: 0.4297 - val_accuracy: 0.9015

15/20 loss: 0.6102 - accuracy: 0.8203 - val_loss: 0.3769 - val_accuracy: 0.9056

16/20 loss: 0.6605 - accuracy: 0.8090 - val_loss: 0.4376 - val_accuracy: 0.8888

17/20 loss: 0.7077 - accuracy: 0.7995 - val_loss: 0.4200 - val_accuracy: 0.8991

18/20 loss: 0.7834 - accuracy: 0.7817 - val_loss: 4.5661 - val_accuracy: 0.6070

19/20 loss: 0.9441 - accuracy: 0.7589 - val_loss: 0.5176 - val_accuracy: 0.8646

20/20 loss: 2.2605 - accuracy: 0.7059 - val_loss: 0.7447 - val_accuracy: 0.8210

Сохраним модель на Google Disk.

In [ ]:
!cp augmentation_model.h5 /GD/'My Drive'/augmentation_model.h5

Подготовим генераторы.

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255)
                            
train_generator = train_datagen.flow_from_directory('to_train',
                                                   target_size = (Pic_Y, Pic_X),
                                                   batch_size = 64,
                                                   class_mode = 'categorical')

Found 81341 images belonging to 9 classes.


In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)
                            
test_generator = test_datagen.flow_from_directory('to_test',
                                                   target_size = (Pic_Y, Pic_X),
                                                   batch_size = 64,
                                                   class_mode = 'categorical')

Found 18690 images belonging to 9 classes.


Подготовим модель с бОльшим количеством слоев на выходе.

In [ ]:
model = models.Sequential()

model.add(layers.Conv2D(filters=32, kernel_size=3, activation="relu", 
                        input_shape = (Pic_Y, Pic_X, 3)))
model.add(layers.MaxPooling2D(pool_size=2))

model.add(layers.Conv2D(filters=64, kernel_size=3, activation="relu"))
model.add(layers.MaxPooling2D(pool_size=2))

model.add(layers.Conv2D(filters=128, kernel_size=3, activation="relu"))
model.add(layers.MaxPooling2D(pool_size=2))

model.add(layers.Conv2D(filters=256, kernel_size=3, activation="relu"))
model.add(layers.MaxPooling2D(pool_size=2))

model.add(layers.Conv2D(filters=256, kernel_size=3, activation="relu"))

model.add(layers.Flatten())

model.add(layers.Dense(64, activation="sigmoid"))

model.add(layers.Dense(9, activation="softmax"))

In [ ]:
model.compile(loss="categorical_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

In [ ]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 124, 222, 32)      896       
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 62, 111, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_16 (Conv2D)          (None, 60, 109, 64)       18496     
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 30, 54, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_17 (Conv2D)          (None, 28, 52, 128)       73856     
                                                                 
 max_pooling2d_14 (MaxPoolin  (None, 14, 26, 128)     

In [ ]:
callbacks = [tf.keras.callbacks.ModelCheckpoint(
        filepath="convnet_with_dense.h5",
        save_best_only=True,
        monitor="val_loss")]

In [ ]:
history = model.fit(train_generator, epochs=25, validation_data=test_generator,
    callbacks=callbacks)

1 - loss: 1.5043 - accuracy: 0.4468 - val_loss: 0.9425 - val_accuracy: 0.6766

2 - loss: 0.6992 - accuracy: 0.7557 - val_loss: 0.5143 - val_accuracy: 0.8239

3 - loss: 0.4488 - accuracy: 0.8479 - val_loss: 0.4289 - val_accuracy: 0.8585

4 - loss: 0.3282 - accuracy: 0.8915 - val_loss: 0.3333 - val_accuracy: 0.8914

5 - loss: 0.2620 - accuracy: 0.9138 - val_loss: 0.2878 - val_accuracy: 0.9074

6 - loss: 0.2211 - accuracy: 0.9269 - val_loss: 0.3207 - val_accuracy: 0.9037

7 - loss: 0.1997 - accuracy: 0.9341 - val_loss: 0.2534 - val_accuracy: 0.9217

8 - loss: 0.1942 - accuracy: 0.9369 - val_loss: 0.3457 - val_accuracy: 0.8903

9 - loss: 0.1966 - accuracy: 0.9364 - val_loss: 0.2605 - val_accuracy: 0.9188

10 - loss: 0.2147 - accuracy: 0.9303 - val_loss: 0.3784 - val_accuracy: 0.8870

11 - loss: 0.2361 - accuracy: 0.9233 - val_loss: 0.3131 - val_accuracy: 0.9056

12 - loss: 0.2876 - accuracy: 0.9068 - val_loss: 0.5544 - val_accuracy: 0.8222

13 - loss: 0.3422 - accuracy: 0.8878 - val_loss: 0.3809 - val_accuracy: 0.8837

14 - loss: 0.4230 - accuracy: 0.8638 - val_loss: 0.9980 - val_accuracy: 0.6919

15 - loss: 0.5579 - accuracy: 0.8151 - val_loss: 0.7966 - val_accuracy: 0.7251

16 - loss: 0.7259 - accuracy: 0.7602 - val_loss: 0.6261 - val_accuracy: 0.8074

17 - loss: 1.0858 - accuracy: 0.6479 - val_loss: 1.5498 - val_accuracy: 0.4666

18 - loss: 1.2996 - accuracy: 0.5581 - val_loss: 1.0833 - val_accuracy: 0.6445

19: loss: 1.7015 - accuracy: 0.4050 - val_loss: 1.8507 - val_accuracy: 0.3426

20: loss: 1.9784 - accuracy: 0.2712 - val_loss: 1.9197 - val_accuracy: 0.2890

21: loss: 1.8431 - accuracy: 0.3364 - val_loss: 1.9322 - val_accuracy: 0.2882

22: loss: 1.8737 - accuracy: 0.3513 - val_loss: 2.0209 - val_accuracy: 0.3074

23: loss: 1.9830 - accuracy: 0.2866 - val_loss: 2.0083 - val_accuracy: 0.2393

24: loss: 2.0157 - accuracy: 0.2559 - val_loss: 2.0380 - val_accuracy: 0.2197

25: loss: 1.9627 - accuracy: 0.2880 - val_loss: 1.8874 - val_accuracy: 0.3392

In [ ]:
!cp convnet_with_dense.h5 /GD/'My Drive'/convnet_with_dense.h5

Добавим еще один полносвязный слой.

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255)
                            
train_generator = train_datagen.flow_from_directory('to_train',
                                                   target_size = (Pic_Y, Pic_X),
                                                   batch_size = 64,
                                                   class_mode = 'categorical')

Found 81341 images belonging to 9 classes.


In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)
                            
test_generator = test_datagen.flow_from_directory('to_test',
                                                   target_size = (Pic_Y, Pic_X),
                                                   batch_size = 64,
                                                   class_mode = 'categorical')

Found 18690 images belonging to 9 classes.


In [ ]:
model = models.Sequential()

model.add(layers.Conv2D(filters=32, kernel_size=3, activation="relu", 
                        input_shape = (Pic_Y, Pic_X, 3)))
model.add(layers.MaxPooling2D(pool_size=2))

model.add(layers.Conv2D(filters=64, kernel_size=3, activation="relu"))
model.add(layers.MaxPooling2D(pool_size=2))

model.add(layers.Conv2D(filters=128, kernel_size=3, activation="relu"))
model.add(layers.MaxPooling2D(pool_size=2))

model.add(layers.Conv2D(filters=256, kernel_size=3, activation="relu"))
model.add(layers.MaxPooling2D(pool_size=2))

model.add(layers.Conv2D(filters=256, kernel_size=3, activation="relu"))

model.add(layers.Flatten())

model.add(layers.Dense(128, activation="sigmoid"))

model.add(layers.Dense(64, activation="sigmoid"))

model.add(layers.Dense(9, activation="softmax"))

In [ ]:
model.compile(loss="categorical_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_11 (Conv2D)          (None, 124, 222, 32)      896       
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 62, 111, 32)      0         
 2D)                                                             
                                                                 
 conv2d_12 (Conv2D)          (None, 60, 109, 64)       18496     
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 30, 54, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 28, 52, 128)       73856     
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 14, 26, 128)     

In [ ]:
callbacks = [tf.keras.callbacks.EarlyStopping(monitor="val_loss",
                                              patience=2),
             tf.keras.callbacks.ModelCheckpoint(
                 filepath="convnet_with_2_dense.h5",
                 save_best_only=True,
                 monitor="val_loss")]

In [ ]:
history = model.fit(train_generator, epochs=25, validation_data=test_generator,
    callbacks=callbacks)

1/25 - loss: 1.6187 - accuracy: 0.4150 - val_loss: 1.0379 - val_accuracy: 0.6503

2/25 - loss: 0.7865 - accuracy: 0.7295 - val_loss: 0.5314 - val_accuracy: 0.8266

3/25 - loss: 0.4723 - accuracy: 0.8427 - val_loss: 0.3801 - val_accuracy: 0.8764

4/25 - loss: 0.3361 - accuracy: 0.8889 - val_loss: 0.3191 - val_accuracy: 0.8965

5/25 - loss: 0.2604 - accuracy: 0.9146 - val_loss: 0.3494 - val_accuracy: 0.8905

6/25 - loss: 0.2203 - accuracy: 0.9268 - val_loss: 0.2424 - val_accuracy: 0.9247

7/25 - loss: 0.1909 - accuracy: 0.9364 - val_loss: 0.2574 - val_accuracy: 0.9203

8/25 - loss: 0.1811 - accuracy: 0.9406 - val_loss: 0.2729 - val_accuracy: 0.9172

In [ ]:
!cp convnet_with_2_dense.h5 /GD/'My Drive'/convnet_with_2_dense.h5

Добавим отступы, чтобы замедлить уменьшение размера и добавить еще одну свёртку.

In [6]:
train_datagen = ImageDataGenerator(rescale = 1./255)
                            
train_generator = train_datagen.flow_from_directory('to_train',
                                                   target_size = (Pic_Y, Pic_X),
                                                   batch_size = 64,
                                                   class_mode = 'categorical')

Found 81341 images belonging to 9 classes.


In [7]:
test_datagen = ImageDataGenerator(rescale = 1./255)
                            
test_generator = test_datagen.flow_from_directory('to_test',
                                                   target_size = (Pic_Y, Pic_X),
                                                   batch_size = 64,
                                                   class_mode = 'categorical')

Found 18690 images belonging to 9 classes.


In [8]:
model = models.Sequential()

model.add(layers.Conv2D(filters=32, kernel_size=3, activation="relu", 
                        padding="same",
                        input_shape = (Pic_Y, Pic_X, 3)))
model.add(layers.MaxPooling2D(pool_size=2))

model.add(layers.Conv2D(filters=64, kernel_size=3, activation="relu",
                        padding="same"))
model.add(layers.MaxPooling2D(pool_size=2))

model.add(layers.Conv2D(filters=128, kernel_size=3, activation="relu",
                        padding="same"))
model.add(layers.MaxPooling2D(pool_size=2))

model.add(layers.Conv2D(filters=256, kernel_size=3, activation="relu",
                        padding="same"))
model.add(layers.MaxPooling2D(pool_size=2))

model.add(layers.Conv2D(filters=256, kernel_size=3, activation="relu",
                        padding="same"))
model.add(layers.MaxPooling2D(pool_size=2))

model.add(layers.Conv2D(filters=512, kernel_size=3, activation="relu"))

model.add(layers.Flatten())

model.add(layers.Dense(128, activation="sigmoid"))

model.add(layers.Dense(64, activation="sigmoid"))

model.add(layers.Dense(9, activation="softmax"))

In [9]:
model.compile(loss="categorical_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 224, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 112, 32)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 63, 112, 64)       18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 31, 56, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 31, 56, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 15, 28, 128)      0

In [11]:
callbacks = [tf.keras.callbacks.EarlyStopping(monitor="val_loss",
                                              patience=2),
             tf.keras.callbacks.ModelCheckpoint(
                 filepath="convnet_with_add_conv_and_2_dense.h5",
                 save_best_only=True,
                 monitor="val_loss")]

In [ ]:
history = model.fit(train_generator, epochs=25, validation_data=test_generator,
    callbacks=callbacks)

1/25 loss: 1.4690 - accuracy: 0.4667 - val_loss: 0.8517 - val_accuracy: 0.7143

2/25 loss: 0.6703 - accuracy: 0.7744 - val_loss: 0.4461 - val_accuracy: 0.8520

3/25 loss: 0.3757 - accuracy: 0.8770 - val_loss: 0.3729 - val_accuracy: 0.8835

4/25 loss: 0.2760 - accuracy: 0.9109 - val_loss: 0.2761 - val_accuracy: 0.9133

5/25 loss: 0.2249 - accuracy: 0.9284 - val_loss: 0.2608 - val_accuracy: 0.9183

6/25 loss: 0.2035 - accuracy: 0.9356 - val_loss: 0.2709 - val_accuracy: 0.9185

7/25 loss: 0.2024 - accuracy: 0.9359 - val_loss: 0.2682 - val_accuracy: 0.9213

In [13]:
!cp convnet_with_add_conv_and_2_dense.h5 /GD/'My Drive'/convnet_with_add_conv_and_2_dense.h5